In [2]:
import qcfractal.interface as ptl
client = ptl.FractalClient(address="localhost:7777", verify=False)

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
pd.set_option('display.max_rows', None)
warnings.filterwarnings("ignore")

In [4]:
%run /home/fastdata/Shared/Masters_Project/My_Classes.ipynb

In [8]:
import inspect

methods_and_docstrings = {
    name: obj.__doc__ for name, obj in inspect.getmembers(ds_be) 
    if callable(obj) and not name.startswith("__")
}

for method, doc in methods_and_docstrings.items():
    print(f"Method: {method}")
    print(f"Description: {doc}")
    print("-" * 40)

NameError: name 'ds_be' is not defined

## $\nu_{TST}$ calculation

 - We need:
     - $I_{x},I_{y}, I_{z}$: principle moments of inertia 
         - We need to use the bond analge snad lengths of the geometrically optimised structures and diagonalise the matrix tensor
     - Symmetry factor $\sigma$
         - Can be found intuitively
     - $T_{peak}$: Monolayer Peak Desorption Temperature
         - ...
         
- We need to do this for:
    - CO
    - $CH_{3}OH$
    - $H_{2}CO$
    - $H_{2}$
    - $NH_{3}$
    - OH
    - $CH_{4}$
    - $CH_{2}OH$
    - $CH_{3}O$
    - $CH_{3}$
    - $H_{2}S$
    - HCl
    - HCOOH
    - HCO
    - HF
    - HNC
    - $NH_{2}$
    - $NHCH_{2}$
    - $C_{2}H_{2}$
    - $H_{2}O$
    - $N_{2}$
    - $HCN$


    

In [423]:
ds_mols  = client.get_collection("OptimizationDataset", "project_molecules")
ds_mols.status(collapse = False)

,blyp-d3bj_def2-svpd,b3lyp-d3bj_def2-svpd,hf-3c-d3bj_MINIX,b3lyp_def2-svpd,hf-3c_MINIX,hf-d3bj_sto-3g,hf_sto-3g,hf_MINIX,mpwb1k-d3bj_def2-tzvp,mpwb1k-d3bj_def2-svpd
C2H2,None,None,None,None,None,None,None,None,COMPLETE,COMPLETE
CO2_CO,None,None,None,None,None,None,None,None,COMPLETE,COMPLETE
H2O_H2O,None,None,None,None,None,None,None,None,COMPLETE,COMPLETE
H2O,None,None,None,None,None,None,None,None,COMPLETE,COMPLETE
CO2_CO2,None,None,None,None,None,None,None,None,COMPLETE,COMPLETE
CH4,None,None,None,None,None,None,None,None,COMPLETE,COMPLETE
CO,None,None,None,None,None,None,None,None,COMPLETE,COMPLETE
CH3OH,None,None,None,None,None,None,None,None,COMPLETE,COMPLETE
CO2,None,None,None,None,None,None,None,None,COMPLETE,COMPLETE
H2CO,None,None,None,None,None,None,None,None,COMPLETE,COMPLETE


## Obtaining the Moments of Interia and Symmetry Factors

In [387]:
import numpy as np


def get_xyz_positions(molecule):
    ds_mols  = client.get_collection("OptimizationDataset", "project_molecules")
    r=ds_mols.get_record(molecule,'mpwb1k-d3bj_def2-tzvp')
    a=(r.get_final_molecule().pretty_print()).split('\n')
    data = [s.split() for s in a 
     if  s.strip() and not s.startswith(('Geometry', 'Center', '-'))][3:]

    cleaned_data=[[d[0]]+ list(map(float,d[1:]))for d in data]
    return cleaned_data
    

def calculate_moments_of_inertia(atoms, coordinates):
    """
    Calculate the principal moments of inertia for a molecule.

    Parameters:
    atoms : list of tuples
        A list of tuples where each tuple contains the atomic symbol and mass of an atom.
    coordinates : numpy array
        A 2D numpy array where each row represents the (x, y, z) coordinates of an atom.

    Returns:
    numpy array
        The principal moments of inertia.
    """
    # Calculate the center of mass
    total_mass = sum(mass for _, mass in atoms)
    coordinates = np.array([coordinates[i][1:] for i in range(len(coordinates))])
    center_of_mass = sum(mass * coordinates[i] for i, (_, mass) in enumerate(atoms)) / total_mass

    # Translate coordinates to the center of mass
    translated_coords = coordinates - center_of_mass

    # Initialize the inertia tensor
    inertia_tensor = np.zeros((3, 3))

    # Calculate the inertia tensor
    for i, (_, mass) in enumerate(atoms):
        x, y, z = translated_coords[i]
        inertia_tensor[0, 0] += mass * (y**2 + z**2)
        inertia_tensor[1, 1] += mass * (x**2 + z**2)
        inertia_tensor[2, 2] += mass * (x**2 + y**2)
        inertia_tensor[0, 1] -= mass * x * y
        inertia_tensor[0, 2] -= mass * x * z
        inertia_tensor[1, 2] -= mass * y * z

    # Symmetrize the inertia tensor
    inertia_tensor[1, 0] = inertia_tensor[0, 1]
    inertia_tensor[2, 0] = inertia_tensor[0, 2]
    inertia_tensor[2, 1] = inertia_tensor[1, 2]

    # Calculate the principal moments of inertia
    principal_moments = np.linalg.eigvalsh(inertia_tensor)

    return principal_moments

# Example usage
# Define atoms as (symbol, mass) and coordinates as (x, y, z)
# atoms = [('C', 1.008), ('O', 16.00), ('H', 1.008)]
# coordinates = np.array([
#     [0.0, 0.0, 0.0],  # Oxygen
#     [0.9572, 0.0, 0.0],  # Hydrogen 1
#     [-0.2399872, 0.927297, 0.0]  # Hydrogen 2
# ])


molecules = {
    "CO": [("C", 12.011), ("O", 15.999)],
    "CH3OH": [("C", 12.011), ("H", 1.008), ("H", 1.008), ("H", 1.008), ("O", 15.999), ("H", 1.008)],
    "H2CO": [("H", 1.008), ("H", 1.008), ("C", 12.011), ("O", 15.999)],
    "H2": [("H", 1.008), ("H", 1.008)],
    "NH3": [("N", 14.007), ("H", 1.008), ("H", 1.008), ("H", 1.008)],
    "OH": [("O", 15.999), ("H", 1.008)],
    "CH4": [("C", 12.011), ("H", 1.008), ("H", 1.008), ("H", 1.008), ("H", 1.008)],
    "CH2OH": [("C", 12.011), ("H", 1.008), ("H", 1.008), ("O", 15.999), ("H", 1.008)],
    "CH3O": [("C", 12.011), ("H", 1.008), ("H", 1.008), ("H", 1.008), ("O", 15.999)],
    "CH3": [("C", 12.011), ("H", 1.008), ("H", 1.008), ("H", 1.008)],
    "H2S": [("H", 1.008), ("H", 1.008), ("S", 32.06)],
    "HCl": [("H", 1.008), ("Cl", 35.45)],
    "HCOOH": [("H", 1.008), ("C", 12.011), ("O", 15.999), ("O", 15.999), ("H", 1.008)],
    "HCO": [("H", 1.008), ("C", 12.011), ("O", 15.999)],
    "HF": [("H", 1.008), ("F", 18.998)],
    "HNC": [("H", 1.008), ("N", 14.007), ("C", 12.011)],
    "NH2": [("N", 14.007), ("H", 1.008), ("H", 1.008)],
    #"NHCH2": [("N", 14.007), ("H", 1.008), ("C", 12.011), ("H", 1.008), ("H", 1.008)],
    "C2H2": [("C", 12.011), ("C", 12.011), ("H", 1.008), ("H", 1.008)],
    "H2O": [("H", 1.008), ("H", 1.008), ("O", 15.999)],
    "N2": [("N", 14.007), ("N", 14.007)],
    "HCN": [("H", 1.008), ("C", 12.011), ("N", 14.007)]
}





list_of_moments_of_inertia=[]
for molecule,atom in molecules.items():
    atoms = atom
    coordinates = get_xyz_positions(molecule)
    moments_of_inertia = calculate_moments_of_inertia(atoms, coordinates)
    d= {'molecule':molecule,'Ix':moments_of_inertia[0],'Iy':moments_of_inertia[1],'Iz':moments_of_inertia[2]}
    list_of_moments_of_inertia=np.append(list_of_moments_of_inertia,d)
    
list_of_moments_of_inertia
    #print(f"Principal moments of inertia (amu·Å²): for {molecule} ", moments_of_inertia)

array([{'molecule': 'CO', 'Ix': 1.1102230246251565e-16, 'Iy': 8.51611254718944, 'Iz': 8.51611254718944},
       {'molecule': 'CH3OH', 'Ix': 3.868063945955981, 'Iy': 19.86910313245124, 'Iz': 20.58201263904359},
       {'molecule': 'H2CO', 'Ix': 2.8785033427780027, 'Iy': 27.5208082565911, 'Iz': 30.3993115993691},
       {'molecule': 'H2', 'Ix': -2.168404344971009e-19, 'Iy': 0.2755535961151959, 'Iz': 0.275553596115196},
       {'molecule': 'NH3', 'Ix': 1.6562979292918527, 'Iy': 1.6563034882218666, 'Iz': 2.647630510062533},
       {'molecule': 'OH', 'Ix': 2.1453653523269864e-33, 'Iy': 0.8831604560405357, 'Iz': 0.8831604560405357},
       {'molecule': 'CH4', 'Ix': 3.147939870017881, 'Iy': 3.1479460913673694, 'Iz': 3.147955175516644},
       {'molecule': 'CH2OH', 'Ix': 2.5310521089347047, 'Iy': 16.44444704820559, 'Iz': 18.97549915714029},
       {'molecule': 'CH3O', 'Ix': 2.423270963571177, 'Iy': 78.53822257173535, 'Iz': 80.96149353530652},
       {'molecule': 'CH3', 'Ix': 1.7401302511850303

In [421]:
df_I_values = pd.DataFrame(data=np.array([[list_of_moments_of_inertia[i]['Ix'] 
                                                               for i in range(len(list_of_moments_of_inertia))],
                                     [list_of_moments_of_inertia[i]['Iy'] 
                                                               for i in range(len(list_of_moments_of_inertia))],
                                      [list_of_moments_of_inertia[i]['Iz'] 
                                                               for i in range(len(list_of_moments_of_inertia))]]).transpose()
                         ,columns=['$I_{x}$','$I_{y}$','$I_{z}$']
                         ,index=[key for key,items in molecules.items()])



symmetry_factors = {
    "CO": 1,      
    "CH3OH": 1,   
    "H2CO": 1,    
    "H2": 2,   
    "NH3": 3,  
    "OH": 1,   
    "CH4": 12, 
    "CH2OH": 1,
    "CH3O": 1, 
    "CH3": 3,  
    "H2S": 2,  
    "HCl": 1,  
    "HCOOH": 1,
    "HCO": 1,  
    "HF": 1,   
    "HNC": 1,  
    "NH2": 2,  
    #"NHCH2": 1,
    "C2H2": 2, 
    "H2O": 2,  
    "N2": 2,   
    "HCN": 1  } 

df_I_values['$\sigma$']= [value for key,value in symmetry_factors.items()]
#df_I_values['$I_{x}$']=np.where(df_I_values['$I_{x}$']<=10e-10, 0,df_I_values['$I_{x}$'])
df_I_values["2D-rotations?"] = ['yes' if i<=10e-10 else 'no' for i in df_I_values['$I_{x}$']]
df_I_values


,$I_{x}$,$I_{y}$,$I_{z}$,$\sigma$,2D-rotations?
CO,1.110223e-16,8.516113,8.516113,1,yes
CH3OH,3.868064e+00,19.869103,20.582013,1,no
H2CO,2.878503e+00,27.520808,30.399312,1,no
H2,-2.168404e-19,0.275554,0.275554,2,yes
NH3,1.656298e+00,1.656303,2.647631,3,no
OH,2.145365e-33,0.883160,0.883160,1,yes
CH4,3.147940e+00,3.147946,3.147955,12,no
CH2OH,2.531052e+00,16.444447,18.975499,1,no
CH3O,2.423271e+00,78.538223,80.961494,1,no
CH3,1.740130e+00,1.740138,3.480268,3,no
